In [94]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [128]:
data = pd.read_csv("data.csv")
data.head()

,year,month,day,anger_score,disgust_score,fear_score,joy_score,sadness_score,anger_count,disgust_count,fear_count,joy_count,sadness_count,close,open,high,low,change
0,2011,4,11,23.678598,1.862068,3.365532,129.260966,22.046661,28,2,4,150,26,12381.11,12380.43,12476.93,12319.35,0.0001
1,2011,4,12,23.436641,1.568504,1.807040,110.107599,28.552481,28,2,2,128,33,12263.58,12381.04,12381.04,12185.09,-0.9500
2,2011,4,13,17.595715,3.441165,3.351268,103.535217,17.206766,21,4,4,121,21,12270.99,12263.73,12372.94,12203.63,0.0006
3,2011,4,14,16.065135,4.921378,5.183304,102.417106,15.457086,19,6,6,121,18,12285.15,12270.24,12332.71,12137.98,0.0012
4,2011,4,15,16.869910,1.555661,3.334624,121.630811,14.709462,20,2,4,141,18,12341.83,12285.45,12402.61,12238.34,0.0046


In [129]:
label = pd.DataFrame(data.iloc[:,-1])
data = data.iloc[:,3:13]

In [130]:
train_data = data.iloc[:-50,:]
test_data = data.iloc[-50:,:]
train_label = label.iloc[:-50,:]
test_label = label.iloc[-50:,:]

In [98]:
scaler_1 = RobustScaler()
scaler_2 = RobustScaler()
scale_data= scaler_1.fit_transform(train_data)
scale_label = pd.DataFrame(scaler_2.fit_transform(pd.DataFrame(train_label)))

In [99]:
scale_data = pd.DataFrame(scale_data,columns=['anger_score', 'disgust_score', 'fear_score',
       'joy_score', 'sadness_score', 'anger_count', 'disgust_count',
       'fear_count', 'joy_count', 'sadness_count'])

In [100]:
#3일뒤 예측
train_label = scale_label.iloc[3:,]
train_data = scale_data.iloc[:-3,:]

In [102]:
test_data = pd.DataFrame(scaler_1.transform(test_data),columns=['anger_score', 'disgust_score', 'fear_score',
       'joy_score', 'sadness_score', 'anger_count', 'disgust_count',
       'fear_count', 'joy_count', 'sadness_count'])
test_label = pd.DataFrame(scaler_2.transform(test_label))

In [103]:
test_data = test_data.iloc[:-3,:]
test_label = test_label.iloc[3:,:]

In [124]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor

In [125]:
GBoost = GradientBoostingRegressor(max_depth=2, learning_rate=0.005,min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [126]:
GBoost.fit(train_data,train_label)

C:\Users\sh2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.005, loss='huber', max_depth=2,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=5, subsample=1.0, verbose=0,
             warm_start=False)

In [127]:
res = GBoost.predict(test_data)
scaler_2.transform(res.reshape(-1,1))

array([[-0.23150617],
       [-0.32320245],
       [-0.39332216],
       [-0.43611592],
       [-0.39332216],
       [-0.39332216],
       [-0.43611592],
       [-0.43611592],
       [-0.42160183],
       [-0.36417628],
       [-0.27876955],
       [-0.32138253],
       [-0.39332216],
       [-0.43611592],
       [-0.39332216],
       [-0.43611592],
       [-0.3659962 ],
       [-0.39332216],
       [-0.32320245],
       [-0.21741505],
       [-0.57432296],
       [-0.32320245],
       [-0.43611592],
       [-0.43611592],
       [-0.43611592],
       [-0.43611592],
       [-0.43611592],
       [-0.32320245],
       [-0.3659962 ],
       [-0.46439558],
       [-0.42160183],
       [-0.39332216],
       [-0.39332216],
       [-0.32138253],
       [-0.43611592],
       [-0.43611592],
       [-0.23332609],
       [-0.43611592],
       [-0.42160183],
       [-0.43611592],
       [-0.21510883],
       [-0.43611592],
       [-0.42160183],
       [-0.43611592],
       [-0.43611592],
       [-0

In [115]:
XGBoost = xgb.XGBRegressor(learning_rate=0.001,n_estimators=5000)
XGBoost.fit(train_data,train_label)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.001, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=5000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [116]:
res = XGBoost.predict(test_data)

In [131]:
test_label

,change
322,-0.9300
323,-0.7900
324,-0.8200
325,0.0047
326,0.0167
327,0.0146
328,-0.0200
329,-0.4900
330,-0.2900
331,-0.7100


In [117]:
scaler_2.transform(res.reshape(-1,1))

array([[ -0.41827145],
       [ -2.950281  ],
       [ -0.99435145],
       [ -1.2365125 ],
       [ -1.0014337 ],
       [ -0.89890766],
       [ -1.3946278 ],
       [ -1.5212314 ],
       [ -0.9428358 ],
       [ -1.1394005 ],
       [ -0.5302758 ],
       [ -0.31085342],
       [ -0.70468074],
       [ -1.466225  ],
       [ -0.8833878 ],
       [ -1.2042979 ],
       [ -0.8470536 ],
       [ -0.67851126],
       [ -2.7009673 ],
       [ -1.3135097 ],
       [ -2.0962284 ],
       [ -0.3353672 ],
       [ -1.3341479 ],
       [ -1.0007073 ],
       [ -1.7626572 ],
       [ -1.4945529 ],
       [ -1.295532  ],
       [ -2.7009673 ],
       [ -2.8094718 ],
       [ -1.4556935 ],
       [ -1.2184819 ],
       [ -0.90442115],
       [ -1.1987966 ],
       [ -0.51102203],
       [ -1.3439208 ],
       [ -1.6187255 ],
       [ -2.9024568 ],
       [ -1.5127535 ],
       [ -1.1924999 ],
       [ -1.1874791 ],
       [ -0.74749637],
       [ -1.089188  ],
       [-11.757088  ],
       [ -1

In [111]:
scaler_2.transform(test_label)

array([[ 1.21637237e-02],
       [ 4.73726730e-02],
       [ 4.12111069e-02],
       [-6.03080302e-02],
       [-1.43932521e+00],
       [-8.52509389e-01],
       [-2.08482261e+00],
       [ 4.79594888e-02],
       [ 3.06807850e-03],
       [ 9.81646044e-03],
       [-1.59408518e-04],
       [-2.36352777e-01],
       [ 7.76260507e-03],
       [-8.52509389e-01],
       [-1.03963225e-03],
       [-1.77671194e-01],
       [ 1.74450661e-02],
       [ 3.94830223e-03],
       [-8.96488212e-02],
       [-1.49800679e+00],
       [-6.76464642e-01],
       [-2.58361606e+00],
       [ 2.09659610e-02],
       [-7.35146225e-01],
       [-5.00419896e-01],
       [-7.46224339e-04],
       [-2.37823052e+00],
       [ 1.86186978e-02],
       [-1.23393967e+00],
       [ 1.01422312e-03],
       [ 5.32408312e-02],
       [ 1.60103895e-03],
       [-1.14591730e+00],
       [ 1.36307633e-02],
       [ 4.27407303e-04],
       [ 4.38517781e-02],
       [ 1.01098684e-02],
       [-8.81850180e-01],
       [ 1.0

ValueError: non-broadcastable output operand with shape (11,1) doesn't match the broadcast shape (11,15)